# 目标：1000MW的电解槽

# 过程：1144MW的机组（1MW的机组只能产生85%~90%的电）

# 结果：不同的风光比例，对应的电解槽在线时长（20%负荷的在线时长，50%负荷的在线时长），费用，占地, 弃电

In [1]:
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly

# 1 导入数据

In [3]:
d4f = pd.read_excel("Solar_Pakistan.xlsx")     #导入csv文件
d1f = pd.read_excel("Wind_Pakistan.xlsx")     #导入csv文件
d4f =d4f.rename(columns={0:"Solar"})
d1f =d1f.rename(columns={0:"Wind"})
#df = df.drop("Unnamed: 0")
df = pd.DataFrame()
df["Solar"] = d4f["Solar"]
df["Wind"] = d1f["Wind"]
df

,Solar,Wind
0,-0.075000,2.954155e+06
1,-0.075000,2.977993e+06
2,-0.075000,2.980643e+06
3,-0.075000,2.990910e+06
4,57.990405,2.950177e+06
...,...,...
8779,-0.075000,2.369153e+06
8780,-0.075000,2.238382e+06
8781,-0.075000,2.194194e+06
8782,-0.075000,2.098510e+06


In [4]:
for i in range(len(df["Solar"])):
    if pd.isnull(df["Solar"].values[i]):
        df_last = df["Solar"][i-1]
        df_next= df["Solar"][i+1]
        df_mean = (df_last+df_next)/2
        df["Solar"].values[i] = df_mean

In [5]:
for i in range(len(df["Wind"])):
    if pd.isnull(df["Wind"].values[i]):
        df_last = df["Wind"][i-1]
        df_next= df["Wind"][i+1]
        df_mean = (df_last+df_next)/2
        df["Wind"].values[i] = df_mean

In [6]:
df

,Solar,Wind
0,-0.075000,2.954155e+06
1,-0.075000,2.977993e+06
2,-0.075000,2.980643e+06
3,-0.075000,2.990910e+06
4,57.990405,2.950177e+06
...,...,...
8779,-0.075000,2.369153e+06
8780,-0.075000,2.238382e+06
8781,-0.075000,2.194194e+06
8782,-0.075000,2.098510e+06


In [7]:


date = pd.DataFrame({"Date":pd.date_range(start = "2017-01-01 04:00:00",periods=8760,freq="3600S")})#加了一列时间
df=pd.concat([df,date],axis=1)
df['Date']=df['Date'].apply(lambda x:pd.to_datetime(x,dayfirst=True))
def gy(s):
    return s.year
def gm(s):
    return s.month
def gd(s):
    return s.day
def gh(s):
    return s.hour
def gmd(s):
    return s.minute
df['year']=df['Date'].apply(lambda x:gy(x))
df['month']=df['Date'].apply(lambda x:gm(x))    #加两个新列：年，月
df['day']=df['Date'].apply(lambda x:gd(x)) 
df['hour']=df['Date'].apply(lambda x:gh(x)) 
df['minute']=df['Date'].apply(lambda x:gmd(x))
df

,Solar,Wind,Date,year,month,day,hour,minute
0,-0.075000,2.954155e+06,2017-01-01 04:00:00,2017.0,1.0,1.0,4.0,0.0
1,-0.075000,2.977993e+06,2017-01-01 05:00:00,2017.0,1.0,1.0,5.0,0.0
2,-0.075000,2.980643e+06,2017-01-01 06:00:00,2017.0,1.0,1.0,6.0,0.0
3,-0.075000,2.990910e+06,2017-01-01 07:00:00,2017.0,1.0,1.0,7.0,0.0
4,57.990405,2.950177e+06,2017-01-01 08:00:00,2017.0,1.0,1.0,8.0,0.0
...,...,...,...,...,...,...,...,...
8779,-0.075000,2.369153e+06,NaT,NaN,NaN,NaN,NaN,NaN
8780,-0.075000,2.238382e+06,NaT,NaN,NaN,NaN,NaN,NaN
8781,-0.075000,2.194194e+06,NaT,NaN,NaN,NaN,NaN,NaN
8782,-0.075000,2.098510e+06,NaT,NaN,NaN,NaN,NaN,NaN


# 3 确定最佳的风光比例

## 3.1 不同风光比例对应的电解槽在线率

In [8]:
print("该例子中的光电峰值为",max(df["Solar"]))
print("该例子中的风电峰值为",max(df["Wind"]))
print("此时的风光比例为：",round(max(df["Wind"])/max(df["Solar"]),2))

该例子中的光电峰值为 188.7276297936627
该例子中的风电峰值为 3000000.0
此时的风光比例为： 15895.92


## 3.1 使风和光的最大值均为1000MW

In [9]:
ratio_wind = 1000/max(df["Wind"])
ratio_solar = 1000/max(df["Solar"])                    
df["Solar"] = df["Solar"].values*ratio_solar
df["Wind"] = df["Wind"].values*ratio_wind 

In [10]:
# 假设1：
#     后续电解槽的满负荷为1000 MW
#     由于机组只能产生85%~90%的电，取87.5%，即需要1144 MW的机组
df["TotalPower"] =df["Solar"].values +df["Wind"].values
np.max(df["TotalPower"])

2000.0

In [11]:
def power1000(x1):
    a2 = pd.DataFrame(x1)
    s1 = np.where(a2>1000)
    a = s1[0]#找出为0 的时刻，即第几个15分钟
    s22 = []
    s = []  # 定义一个空列表
    for i in a:
        if len(s) == 0 or s[-1] + 1 == i:  # 判断，如果是空列表或者列表中最后一个值加1等于当前值
            s.append(i)  # 写入列表
        else:  # 不满足以上条件
            if len(s) >= 1:  # 判断长度是否大于1， 大于1则打印出来
                s22.append(s)
            s = [i]  # 给s重新复制为i,最后一个值，可以放弃
    if len(s) >= 1:  # 最后一轮，判断是否是连续值，是的话就打印
        s22.append(s)
    
    fefew = []
    for ko in s22:
        bg = (len(ko))
        fefew.append(bg)
    
    sumpower=sum(fefew)
    return(sumpower)
def power100(x1):
    a2 = pd.DataFrame(x1)
    s1 = np.where(a2>100)
    a = s1[0]#找出为0 的时刻，即第几个15分钟
    s22 = []
    s = []  # 定义一个空列表
    for i in a:
        if len(s) == 0 or s[-1] + 1 == i:  # 判断，如果是空列表或者列表中最后一个值加1等于当前值
            s.append(i)  # 写入列表
        else:  # 不满足以上条件
            if len(s) >= 1:  # 判断长度是否大于4， 大于4则打印出来，   即连续一个小时
                s22.append(s)
            s = [i]  # 给s重新复制为i,最后一个值，可以放弃
    if len(s) >= 1:  # 最后一轮，判断是否是连续值，是的话就打印
        s22.append(s)
    
    fefew = []
    for ko in s22:
        bg = (len(ko))
        fefew.append(bg)
    
    sumpower=sum(fefew)
    return(sumpower)

def power200(x1):
    a2 = pd.DataFrame(x1)
    s1 = np.where(a2>200)
    a = s1[0]#找出为0 的时刻，即第几个15分钟
    s22 = []
    s = []  # 定义一个空列表
    for i in a:
        if len(s) == 0 or s[-1] + 1 == i:  # 判断，如果是空列表或者列表中最后一个值加1等于当前值
            s.append(i)  # 写入列表
        else:  # 不满足以上条件
            if len(s) >= 1:  # 判断长度是否大于1， 大于1则打印出来
                s22.append(s)
            s = [i]  # 给s重新复制为i,最后一个值，可以放弃
    if len(s) >= 1:  # 最后一轮，判断是否是连续值，是的话就打印
        s22.append(s)
    
    fefew = []
    for ko in s22:
        bg = (len(ko))
        fefew.append(bg)
    
    sumpower=sum(fefew)
    return(sumpower)

def power300(x1):
    a2 = pd.DataFrame(x1)
    s1 = np.where(a2>300)
    a = s1[0]#找出为0 的时刻，即第几个15分钟
    s22 = []
    s = []  # 定义一个空列表
    for i in a:
        if len(s) == 0 or s[-1] + 1 == i:  # 判断，如果是空列表或者列表中最后一个值加1等于当前值
            s.append(i)  # 写入列表
        else:  # 不满足以上条件
            if len(s) >= 1:  # 判断长度是否大于1， 大于1则打印出来
                s22.append(s)
            s = [i]  # 给s重新复制为i,最后一个值，可以放弃
    if len(s) >= 1:  # 最后一轮，判断是否是连续值，是的话就打印
        s22.append(s)
    
    fefew = []
    for ko in s22:
        bg = (len(ko))
        fefew.append(bg)
    
    sumpower=sum(fefew)
    return(sumpower)

def power400(x1):
    a2 = pd.DataFrame(x1)
    s1 = np.where(a2>400)
    a = s1[0]#找出为0 的时刻，即第几个15分钟
    s22 = []
    s = []  # 定义一个空列表
    for i in a:
        if len(s) == 0 or s[-1] + 1 == i:  # 判断，如果是空列表或者列表中最后一个值加1等于当前值
            s.append(i)  # 写入列表
        else:  # 不满足以上条件
            if len(s) >= 1:  # 判断长度是否大于1， 大于1则打印出来
                s22.append(s)
            s = [i]  # 给s重新复制为i,最后一个值，可以放弃
    if len(s) >= 1:  # 最后一轮，判断是否是连续值，是的话就打印
        s22.append(s)
    
    fefew = []
    for ko in s22:
        bg = (len(ko))
        fefew.append(bg)
    
    sumpower=sum(fefew)
    return(sumpower)

def power500(x1):
    a2 = pd.DataFrame(x1)
    s1 = np.where(a2>500)
    a = s1[0]#找出为0 的时刻，即第几个15分钟
    s22 = []
    s = []  # 定义一个空列表
    for i in a:
        if len(s) == 0 or s[-1] + 1 == i:  # 判断，如果是空列表或者列表中最后一个值加1等于当前值
            s.append(i)  # 写入列表
        else:  # 不满足以上条件
            if len(s) >= 1:  # 判断长度是否大于1， 大于1则打印出来
                s22.append(s)
            s = [i]  # 给s重新复制为i,最后一个值，可以放弃
    if len(s) >= 1:  # 最后一轮，判断是否是连续值，是的话就打印
        s22.append(s)
    
    fefew = []
    for ko in s22:
        bg = (len(ko))
        fefew.append(bg)
    
    sumpower=sum(fefew)
    return(sumpower)

In [12]:
%%time
#from wind_solar import power100,power200,power300,power400,power500
# 引入的几个函数中，已经将时间频率由15分钟改为1小时

x1 = []
onlinetime100 = []
onlinetime200 = []
onlinetime300 = []
onlinetime400 = []
onlinetime500 = []
onlinetime1000 = []

onlinerate100 = []
onlinerate200 = []
onlinerate300 = []
onlinerate400 = []
onlinerate500 = []
onlinerate1000 = []
for i in np.arange(0.1,10.1,0.1):
    x1.append(i)
    solar1 = 1144/(1+i)           #光机组
    wind1  = 1144-solar1          #风机组
    solar2 = (df["Solar"].values/max(df["Solar"].values))*solar1   
    wind2 = ((df["Wind"].values)/max(df["Wind"].values))*wind1
    
    total_power= solar2+wind2
    bili = (max(solar2)+max(wind2))/max(total_power)
    solar2 = bili*solar2
    wind2 = bili*wind2
    total_power = solar2+wind2
    
    a2 = pd.DataFrame(total_power)
    
    time1 = power100(a2)
    onlinerate100.append(time1/8784)   #2020年有366天
    onlinetime100.append(time1)       #电解槽在线时间(>10%)
    
    time2 = power200(a2)
    onlinerate200.append(time2/8784)
    onlinetime200.append(time2)       #电解槽在线时间(>20%)
    
    time3 = power300(a2)
    onlinerate300.append(time3/8784)
    onlinetime300.append(time3)       #电解槽在线时间(>30%)
    
    time4 = power400(a2)
    onlinerate400.append(time4/8784)
    onlinetime400.append(time4)       #电解槽在线时间(>20%)
    
    time5 = power500(a2)
    onlinerate500.append(time5/8784)
    onlinetime500.append(time5)       #电解槽在线时间(>50%)
    
    time6 = power1000(a2)
    onlinerate1000.append(time6/8784)
    onlinetime1000.append(time6)       #电解槽在线时间(>100%)

a32=np.argmax(onlinetime200)/10 +0.1
a33=max(onlinetime200)*100/8784      #除以8784即为在线率，乘以100是下一行中有个%
print("20%电解槽负荷最大的点的风光比例为：",round(a32,1),",","此时的在线率为",round(a33,1),"%")

fig2 = go.Figure()
trace1 = go.Scatter(x=x1,y=onlinerate100,name='电解槽负荷(>10%)')
trace2 = go.Scatter(x=x1,y=onlinerate200,name='电解槽负荷(>20%)')
trace3 = go.Scatter(x=x1,y=onlinerate300,name='电解槽负荷(>30%)')
trace4 = go.Scatter(x=x1,y=onlinerate400,name='电解槽负荷(>40%)')
trace5 = go.Scatter(x=x1,y=onlinerate500,name='电解槽负荷(>50%)')
trace6 = go.Scatter(x=x1,y=onlinerate1000,name='电解槽负荷(>100%)')
fig2 = go.Figure(data=[trace1,trace2,trace3,trace4,trace5,trace6])
fig2.update_layout(title = "1000MW不同风光比例对应的电解槽在线时率",
                   xaxis_title='风光比例',yaxis=dict(title="在线率/%"))
fig2.update_layout(xaxis_type="log")
plotly.offline.plot(fig2, filename='Chart_'+str(7)+'.html',auto_open=False)
fig2.layout.yaxis.tickformat = ',.0%'   
fig2.show()
print("在线率定义：总功率大于某个值的时长/一年")

20%电解槽负荷最大的点的风光比例为： 2.1 , 此时的在线率为 77.3 %


在线率定义：总功率大于某个值的时长/一年
Wall time: 4.27 s


## 3.2 不同风光比例对应的弃电量

In [13]:
x1 = []
curtailed_power = []
for i in np.arange(0.1,10.1,0.1):
    x1.append(i)
    solar1 = 1144/(1+i)           #光机组
    wind1  = 1144-solar1          #风机组

    solar2 = (df["Solar"].values/max(df["Solar"].values))*solar1   
    wind2 = ((df["Wind"].values)/max(df["Wind"].values))*wind1
    
    total_power= solar2+wind2
    bili = (max(solar2)+max(wind2))/max(total_power)
    solar2 = bili*solar2
    wind2 = bili*wind2
    total_power = solar2+wind2

    a2 = pd.DataFrame(total_power)
    
    s1 = np.where(a2<1000*0.2)          #小于200的弃掉
    s3 = np.where(a2>1000)              #大于1000的弃掉
    
    a9 =s3[0]
    b2=[]
    for j in a9:
        b1 = total_power[j]
        b22=b1-1000
        b2.append(b22)
    
    a3 = s1[0]
    a4=[]
    for i1 in a3:
        a5 = total_power[i1]
        a4.append(a5)
        
    a6 = sum(a4)
    b3  = sum(b2)
    b4 = a6+b3
    a7 = sum(total_power)
    a8 = b4/a7
    curtailed_power.append(a8)
    
a322=np.argmin(curtailed_power)/10 +0.1
print("弃电量最少的比例为：",round(a322,1))

fig22 = go.Figure()
trace12 = go.Scatter(x=x1,y=curtailed_power,marker={"color": "#f06464"})
fig22 = go.Figure(data=[trace12])
fig22.update_layout(title = "不同风光比例对应的弃电占比（电解槽的最小负荷为200MW情况下）",
                    xaxis_title='风光比例',
                    yaxis=dict(title="弃电占比"))
fig22.layout.yaxis.tickformat = ',.0%'        
fig22.update_layout(xaxis_type="log")
fig22.show()

弃电量最少的比例为： 2.2


## 3.3 结论：最佳风光比例

In [14]:
print("从电解槽在线率来看，最佳的风光比例为：",round(a32,1))
print("从弃电占比来看，最佳的风光比例为：",round(a322,1))   

从电解槽在线率来看，最佳的风光比例为： 2.1
从弃电占比来看，最佳的风光比例为： 2.2


In [15]:

solar1 = 1200/(1+a32)           #光机组
wind1  = 1200-solar1          #风机组
solar2 = (df["Solar"].values/max(df["Solar"].values))*solar1   
wind2 = ((df["Wind"].values)/max(df["Wind"].values))*wind1
total_power= solar2+wind2
df["Solar"] = solar2
df["Wind"]  =wind2
df["TotalPower"] =total_power

In [16]:
print(max(df["Solar"]))
print(max(df["Wind"]))
print(max(df["TotalPower"]))

387.09677419354836
812.9032258064517
1200.0


# 保存文件

In [17]:
df.to_excel("TotalPower.xlsx")

###  电力小于20%的点

In [18]:
def power20(x1):
    a2 = pd.DataFrame(x1)
    a4 = np.percentile(df["TotalPower"].values,98)*0.2
    s1 = np.where(a2<200)
    a = s1[0]#找出为0 的时刻，即第几个15分钟
    s22 = []
    s = []  # 定义一个空列表
    for i in a:
        if len(s) == 0 or s[-1] + 1 == i:  # 判断，如果是空列表或者列表中最后一个值加1等于当前值
            s.append(i)  # 写入列表
        else:  # 不满足以上条件
            if len(s) >= 1:  # 判断长度是否大于1， 大于1则打印出来
                s22.append(s)
            s = [i]  # 给s重新复制为i,最后一个值，可以放弃
    if len(s) >= 2:  # 最后一轮，判断是否是连续值，是的话就打印
        s22.append(s)
    
    fefew = []
    for ko in s22:
        bg = (len(ko))
        fefew.append(bg)
        
    nj = []  
    for i11 in fefew:
        if i11 > 5:
            nj.append(i11)
    pb = sum(nj)
    
    nj2 = []  
    for i1 in fefew:
        if i1 > 2:
            nj2.append(i1)
    pb2 = sum(nj2)

    print("总功率小于20%的总时长为：",sum(fefew),"小时")
    print("总功率小于20%,并且持续时长大于2小时的总时长为：",pb2,"小时")
    print("总功率小于20%,并且持续时长大于5小时的总时长为：",pb,"小时")
    
    print("总功率小于20%的最大时长为：",max(fefew),"小时")
    print("总功率小于20%的最小时长为：",min(fefew),"小时")
    print("总功率小于20%的平均时长为：",round(np.mean(fefew),2),"小时")
    gew = []
    for il in s22:
        ge = il[0]
        gew1=ge
        gew.append(gew1)
    fig1 = px.scatter( x=gew,y=fefew,size = fefew,color = fefew)
    fig1.update_layout(title = '总功率小于20%的时间分布',
                       coloraxis = {'colorscale':'Turbo'},
                       xaxis = dict(title=dict(text='小时(2020年)',font_size=15)),#横轴标题
                       yaxis = dict(title=dict(text='持续时长/小时',font_size=15)))#纵轴标题
    fig1.show()
    fig2 = px.box(y=fefew)
    fig2.update_layout(title = ' ',      
                       xaxis = dict(title=dict(text=' 总功率小于20%的时间段分散情况',font_size=15)),#横轴标题
                       yaxis = dict(title=dict(text='持续时长/小时',font_size=15)))#纵轴标题
    fig2.show()
a1 = df["TotalPower"]        #柴油发电机价格 应急发电机
power20(a1)

总功率小于20%的总时长为： 1890 小时
总功率小于20%,并且持续时长大于2小时的总时长为： 1746 小时
总功率小于20%,并且持续时长大于5小时的总时长为： 1431 小时
总功率小于20%的最大时长为： 44 小时
总功率小于20%的最小时长为： 1 小时
总功率小于20%的平均时长为： 6.34 小时


In [19]:

fig8 = go.Figure()
trace12 = go.Violin(y=solar2,name = "Solar")
trace13 = go.Violin(y=wind2,name = "Wind")
trace14 = go.Violin(y=total_power,name = "TotalPower")
fig8 = go.Figure(data=[trace12,trace13,trace14])
fig8.update_layout(title = "3.3 风光分布小提琴图",
                    xaxis_title=' ',
                    yaxis=dict(title="功率",side="left"))
fig8.show()